# JavaScript::D3 

-----

## Load packages

Here we load some packages that are used to generate, summarize, and modify datasets:

In [1]:
use Data::Generators;
use Data::Reshapers;
use Data::Summarizers;
use Data::ExampleDatasets;

This loads the ["JavaScript::D3"](https://github.com/antononcube/Raku-JavaScript-D3) package:

In [2]:
use JavaScript::D3;

------

## Setup

Here we use a JavaScript cell that allows the visualization of with [D3.js](https://d3js.org) in Jupyter notebooks:


In [3]:
%%javascript
require.config({ 
     paths: { 
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {   
     console.log(d3);
});

The code above can be obtained with the package function `js-d3-config`:

In [4]:
js-d3-config()

require.config({
     paths: {
     d3: 'https://d3js.org/d3.v7.min'
}});

require(['d3'], function(d3) {
     console.log(d3);
});


Verify that a D3 plot is obtained via a JavaScript cell (taken from [SF1]):

In [5]:
%%javascript
(function(element) { require(['d3'], function(d3) {   
        var data = [1, 2, 4, 8, 16, 8, 4, 2, 1]

        var svg = d3.select(element.get(0)).append('svg')
            .attr('width', 400)
            .attr('height', 200);
        svg.selectAll('circle')
            .data(data)
            .enter()
            .append('circle')
            .attr("cx", function(d, i) {return 40 * (i + 1);})
            .attr("cy", function(d, i) {return 100 + 30 * (i % 3 - 1);})
            .style("fill", "#1570a4")
            .transition().duration(2000)
            .attr("r", function(d) {return 2*d;})
        ;
}) })(element);

-------

## ListPlot

In [20]:
%% > js
say js-d3-list-plot(random-real(12,1000), height=>500, background=>'rgb(10,220,220)', color=>'yellow', title => "Yellow points");

In [22]:
%% js
my @ds2 = random-tabular-dataset(120, <x y>, 
    generators=> { 
        x => { random-variate(NormalDistribution.new(30,12), $_)}, 
        y => { random-real(400, $_) }});
say dimensions(@ds2);

js-d3-list-plot(@ds2, height=>500, background=>'white', color=>'orange');

(120 2)


------

## ListLinePlot

### Single-line 

In [23]:
%% js
js-d3-list-line-plot(@ds2.sort({ $_<x> }), 
height=>500, 
background=>'white', color=>'lightblue', 
title => "List line plot",
x-axis-label=>'X coordinates',
y-axis-label=>'Y coordinates',
margins => %(top=>80, left=>60)
);

### Multi-line

In [9]:
my @dsXYG = random-tabular-dataset(400, <x y group>,
        generators => { x => { random-real(100, $_) },
                        y => { random-real(10, $_) },
                        group => <a b c d> });
                        
@dsXYG = @dsXYG.map( -> $r { given $r<group> {
    when 'a' { $r<y> = 1.2*$r<x> + $r<y> }
    when 'b' { $r<y> = 5*sqrt($r<x>) + $r<y>}
    when 'c' { $r<y> = $r<y> - $r<x>/3 + 50 }
  };
  $r
});
say dimensions(@dsXYG);
say deduce-type(@dsXYG);

(400 3)
Vector(Struct([group, x, y], [Str, Num, Num]), 400)


In [24]:
%%js
js-d3-list-line-plot(@dsXYG.sort({ $_<x> }))

-------

##  Bar chart

In [25]:
my @dsTest2 = random-tabular-dataset(12, <Country Value>, generators=>{Country => <USA UK UA RU CH BG FR GE ES PL DK SW>, Value => { random-real(100, $_) } }  );
records-summary(@dsTest2);
dimensions(@dsTest2)

+------------------------------+--------------+
| Value                        | Country      |
+------------------------------+--------------+
| Min    => 11.767339338642335 | USA     => 3 |
| 1st-Qu => 42.16790005962202  | FR      => 2 |
| Mean   => 61.14611669094628  | UK      => 2 |
| Median => 56.449030165005354 | GE      => 1 |
| 3rd-Qu => 87.58471048793152  | UA      => 1 |
| Max    => 92.60916445487334  | SW      => 1 |
|                              | PL      => 1 |
|                              | (Other) => 1 |
+------------------------------+--------------+


(12 2)

In [27]:
%% js
js-d3-bar-chart(rename-columns(@dsTest2,"Country"=>"Label"))

In [28]:
%% js
js-d3-bar-chart(random-real(120,45), width=>1000)

------

## Histogram

In [29]:
%% > js
say js-d3-histogram(
    random-variate(NormalDistribution.new(120,10), 500), 
    height=>500, 
    background=>'rgb(50,56,65)', 
    title=>'Normal distribution example',
    x-axis-label=>'random value',
    y-axis-label=>'counts', margins => {top=>120} );

------

## Bubble chart

In [30]:
my $nPoints = 100;
my @arr3d = transpose( (random-real(12, $nPoints), random-real(12, $nPoints), random-real(12, $nPoints)) );
deduce-type(@arr3d)

Vector(Vector(Atom((Numeric)), 3), 100)

In [31]:
%%js 
js-d3-bubble-chart(@arr3d, 
color=>'rgb(255,180,0)', 
background=>'rgb(50,56,65)', 
tooltip=>Whatever)

In [34]:
my @ds3DGroups = random-tabular-dataset(100, <x y z group>, 
generators => { x => { random-real(20, $_) }, 
                y => { random-variate(NormalDistribution.new(200,50), $_) },
                z => { random-variate(NormalDistribution.new(20,12), $_) },
                group => <aspirin biscuit cookie>
              } );
records-summary(@ds3DGroups, field-names=><x y z group>);
say deduce-type(@ds3DGroups);

+------------------------------+------------------------------+------------------------------+---------------+
| x                            | y                            | z                            | group         |
+------------------------------+------------------------------+------------------------------+---------------+
| Min    => 0.1049439235383387 | Min    => 86.75976020794309  | Min    => -5.328741149205726 | cookie  => 36 |
| 1st-Qu => 5.720064707601877  | 1st-Qu => 167.94535392182877 | 1st-Qu => 12.071250271499899 | biscuit => 32 |
| Mean   => 10.699604398742917 | Mean   => 198.3612382606583  | Mean   => 21.714907395744703 | a       => 32 |
| Median => 11.379574647571303 | Median => 202.48631224145953 | Median => 21.765053130663794 |               |
| 3rd-Qu => 15.49563351251939  | 3rd-Qu => 228.94482706777427 | 3rd-Qu => 30.236005441359588 |               |
| Max    => 19.99253260567572  | Max    => 338.42171926359686 | Max    => 52.095915994450664 |               |
+

In [35]:
%%js
js-d3-bubble-chart(@ds3DGroups, 
x-axis-label=>'x coordinates',
y-axis-label=>'Normal distribution', 
plot-label=>'Bubble chart over groups',
background=>'rgb(50,56,65)', 
margins => %(left=>10, top=>60),
opacity=>0.5):tooltip:legends

-------

## References

[OV1] Olivia Vane, ["D3 JavaScript visualisation in a Python Jupyter notebook"](https://livingwithmachines.ac.uk/d3-javascript-visualisation-in-a-python-jupyter-notebook), (2020), [livingwithmachines.ac.uk](https://livingwithmachines.ac.uk).

[SF1] Stefaan Lippens, [Custom D3.js Visualization in a Jupyter Notebook](https://www.stefaanlippens.net/jupyter-custom-d3-visualization.html), (2018), [stefaanlippens.net](https://www.stefaanlippens.net).